<a href="https://colab.research.google.com/github/kridtapon/Guppy-DMI-Accelerator/blob/main/Guppy_DMI_Accelerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.4 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Guppy MMA
def calculate_gmma(df):
    short_periods = range(3, 16, 3)  # Updated short periods
    long_periods = range(30, 51, 5)  # Updated long periods

    for period in short_periods:
        df[f'GMMA_Short_{period}'] = df['Close'].ewm(span=period).mean()

    for period in long_periods:
        df[f'GMMA_Long_{period}'] = df['Close'].ewm(span=period).mean()

    return df

# Function to calculate DMI & ADX
def calculate_dmi(df, period=14):
    df['TR'] = np.maximum(df['High'] - df['Low'],
                           np.maximum(abs(df['High'] - df['Close'].shift(1)),
                                      abs(df['Low'] - df['Close'].shift(1))))
    df['ATR'] = df['TR'].rolling(window=period).mean()

    df['+DM'] = np.where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']),
                         np.maximum(df['High'] - df['High'].shift(1), 0), 0)
    df['-DM'] = np.where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)),
                         np.maximum(df['Low'].shift(1) - df['Low'], 0), 0)

    df['+DI'] = (df['+DM'].rolling(window=period).mean() / df['ATR']) * 100
    df['-DI'] = (df['-DM'].rolling(window=period).mean() / df['ATR']) * 100
    df['DX'] = (abs(df['+DI'] - df['-DI']) / abs(df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].rolling(window=period).mean()

    return df

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Guppy MMA
df = calculate_gmma(df)

# Calculate DMI & ADX
df = calculate_dmi(df, period=14)

# Define Entry and Exit signals
short_gmma_cols = [f'GMMA_Short_{p}' for p in range(3, 16, 3)]  # Updated short periods
long_gmma_cols = [f'GMMA_Long_{p}' for p in range(30, 51, 5)]   # Updated long periods

df['GMMA_Compression'] = df[short_gmma_cols].mean(axis=1) < df[long_gmma_cols].mean(axis=1)
df['ADX_Strong'] = df['ADX'] > 25
df['Bullish_Crossover'] = df['+DI'] > df['-DI']
df['Bearish_Crossover'] = df['-DI'] > df['+DI']

df['Entry'] = df['GMMA_Compression'] & df['ADX_Strong'] & df['Bullish_Crossover']
df['Exit'] = df['GMMA_Compression'] & df['ADX_Strong'] & df['Bearish_Crossover']

# Filter data for backtesting period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           343623.994939
Total Return [%]                       243.623995
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                         4614.3124
Max Drawdown [%]                        41.814347
Max Drawdown Duration                       612.0
Total Trades                                   15
Total Closed Trades                            14
Total Open Trades                               1
Open Trade PnL                       53872.963388
Win Rate [%]                            35.714286
Best Trade [%]                         148.268994
Worst Trade [%]                        -12.362744
Avg Winning Trade [%]                   43.458045
Avg Losing Trade [%]                    -4.170153
